# An introduction to Bidirectional Encoder Representations from Transformers (BERT)

This presentation is inspired by the illustrations and contents from [Jacob Devlin](https://nlp.stanford.edu/seminar/details/jdevlin.pdf) and [Jay Alammar](http://jalammar.github.io/).

The original paper published by Google researchers: [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)

## Introduction

* Introduction of BERT (2018) started a new era in Natural Language Processing
* Known as the NLP's ImageNet moment (ImageNet had a huge impact on Computer Vision and learning
* BERT is 'deeply bidirectional', whereas ELMo is known to be 'shallowly bidirectional' and OpenAI GPT is 'unidirectional'
* Already trained on vast amount of data by Google and made available for Transfer Learning (fine tuning)
* Built upon many existing ideas, works and their weaknesses. Including ELMo, OpenAI Transformer based model: GPT and Google's Semi-supervised Sequence Learning
* We already covered ELMo, but here let's jump to BERT

**Trained BERT is available for fune tuning and using on various downstream tasks**

<img src="http://jalammar.github.io/images/bert-transfer-learning.png" alt="BERT 2 steps of train and fune tuning" width="80%"/>

The following figure shows the fine tuning step with more details.

<img src="http://jalammar.github.io/images/BERT-classification-spam.png" alt="BERT fune tuning" width="80%"/>

We only need to train the FFNN+Softmax layer and apply minimal fine tuning changes on the BERT model weights.

## BERT's Architecture

BERT is a multi-layer bidirectional transformer encoder model and the paper presents two model sizes: BERT Base and BERT Large.

* BERT Base has 12 layers of encoders, 768 hidden units and 12 attention heads with a total of 110 million parameters to train
* BERT Large has 24 layers of encoders, 1024 hidden units and 16 attention heads with a total of 340 million parameters to train


<img src="http://jalammar.github.io/images/bert-base-bert-large-encoders.png" alt="BERT architecture overview" width="80%"/>

### BERT's input

To make BERT be able to handle a variety of input representations (for the sake of compatibility with downstream tasks), its input could be a sequence of tokens (sentence, paragraph, etc.) or a pair of sequences of tokens, e.g. <Question, Answer>.


<img src="http://jalammar.github.io/images/bert-encoders-input.png" alt="BERT input" width="80%"/>

The encoder layer is almost identical to what was originally described in the 'Attention is All You Need' paper that introduced `Transformers`

The first input token, `<CLS>` is a special token that represents the type of the task we want the BERT to work on, in this case Classification. If BERT is being trained on a pair sentence based task <sentence1, sentence2> then the two parts are separated via a special token `<SEP>`.



#### Input representation

BERT applies **token embedding**, **Segment Embedding** and **Position Embedding** on the input tokens to form its final word representation to feed the encoders. The final representation vector dimension must match the BERT's expected dimenstion, e.g. 768 for BERT Base.

<img src="https://miro.medium.com/max/788/0*m_kXt3uqZH9e7H4w.png" alt="BERT input representation" width="80%"/>

As depicted above, the results of all three steps are summed to form the final input representation for BERT. Let's quickly go through each one.

---

**Token Embedding:**

BERT uses [WordPiece Embeddings from the Google’s Neural Machine Translation System introduced in 2016](https://arxiv.org/pdf/1609.08144.pdf) as the input **token embedding**.

`WordPiece` is a tokenization method that tries to keep the vocabulary size limited, while being able to generate decent and accurate embedding vectors for out-of-vocabulary words as well.
WordPiece allows BERT to have a limited vocabulary size of 30522 unique words which is vital for lowering the training time.

We won't go through the details here, but `WordPiece` is a language-model likelihood maximization (optimization) problem and the goal is to select a minimal set of WordPieces from a corpus such that the corpus can be best described with them.

**Segment Embedding:**

As mentioned before, BERT's input could be a pair of sentences to allow for compatibility with various downstream tasks that require dual input, e.g. question answering. If all of the input tokens belong to one sentence, i.e. there is no dual input, then the segment embedding vector is simply all zeros.

**Position Embedding:**

Similar to what we had in `Transformers`, position embedding helps BERT's stack of encoders know about the absolute and relative position of each token they're processing.

**Note that BERT's output is simply a corresponding vector for each input token that matches the hidden layer dimenstion (768 in BERT Base). This output could further be used for various downstream tasks.**

<img src="http://jalammar.github.io/images/bert-classifier.png" alt="BERT output" width="80%"/>

Remember the encoders from transformers? That's it! BERT's architecture is simply a stack of these encoders modified on the input and output sides to be compatible with BERT's pretraining and fine-tuning.

<img src="https://jalammar.github.io/images/t/transformer_resideual_layer_norm.png" alt="Transformer encoder used in BERT" width="80%"/>

### BERT's tasks

**BERT has two tasks: 1. Masked Language Model (MLM) 2. Next Sentence Prediction (NSP)**

#### 1. Masked LM

* A bidirectional language model that 'deeply' considers context
* Randomly, mask 15% of the input tokens (after tokenization process) from the input to enforce the bidirectional nature
* Unlike Auto-Encoders, BERT only predicts the masked words rather than reconstructing the whole input

<img src="http://jalammar.github.io/images/BERT-language-modeling-masked-lm.png" alt="MLM in BERT" width="80%"/>

As the above figure suggests, the output vectors **corresponding to the masked tokens** are fed into a FFNN+Softmax for training. We simply don't care about output vectors corresponding to unmasked input tokens.

After FFNN+Softmax we compare the actual BERT's output with the desired output (i.e. the original masked word) and use cross-entropy loss to train the model upon its mistakes.

**How to use this for fine tuning? I don't want to `mask` my input tokens!**

Well, only 80% of those 15% selected tokens are actually 'masked' and replaced by special token [MASK]. 10% of the time they are unchanged and 10% of the time they are replaced by another token!
By doing so over and over again, BERT gets used to input sequences **without** [MASK] tokens and can generate desired and accurate output vectors.

#### 2. Next Sentence Prediction (NSP)

* To learn relationships between sentences
* Given a wikipedia document as first sentence, and a question as the second sentence, can we answer the latter using the former?
* Given a question as the first sentence, and an answer as the second sentence, does the latter correctly answer the former?
* Given two sentences as input, does the second sentence actually come after the first one while making sense?
* All of which while **masking** the input as well: **only during the training phase albeit**

<img src="http://jalammar.github.io/images/bert-next-sentence-prediction.png" alt="NSP in BERT" width="80%"/>

### Fine-Tuning BERT

BERT is pre-trained on Wikipedia (2.5B words) and BookCorpus (800M words). Training such a huge and deep model, requires huge data as well and is not something that we want to do every time we have a downstream task to perform. Hence, **Fine-Tuning** comes to play.

**Training each of the BERT Base and BERT Large took 4 days to complete, with Google's TPU hardware!**

Since BERT's input format is very flexible, many downstream tasks could easily fit in for fine-tuning. Let's look at some of them.


<img src="http://jalammar.github.io/images/bert-tasks.png" alt="BERT downstream fine-tuning tasks" width="100%"/>

**Yeah, my question too! What are these tasks?**

These tasks are included in [The General Language Understanding Evaluation (GLUE) benchmark](https://arxiv.org/pdf/1804.07461.pdf) as a diverse collection for Natural Language Processing. For each GLUE task, BERT fine tunes its final classification layer weights, and in some cases where there's not enough data for a good result, BERT runs some random re-training epochs to fine-tune deeper on some of the hidden layer weights.

1. Multi-Genre Natural Language Inference (MNLI): A classification task. Given a pair of sentences, the goal is to predict whether the second sentence is an entailment, contradiction, or neutral with respect to the first one.

2. Quora Question Pairs (QQP): A binary classification task. Determine if two questions asked on Quora are semantically equivalent.

3. Question Natural Language Inference (QNLI): A Q&A task converted into a binary classification task. The positive examples are (question, sentence) pairs which do contain the correct answer, and the negative examples are (question, sentence) from the same paragraph which do not contain the answer.

4. The Stanford Sentiment Treebank (SST-2): A single sentence (meaning that there is no [SEP] token) binary classification task. Simply sentiment analysis on movie reviews sentences annotated by humans.

5. The Corpus of Linguistic Acceptability (CoLA): A single sentence (meaning that there is no [SEP] token) binary classification task. Predict whether an English sentence is linguistically “acceptable” or not.

6. The Semantic Textual Similarity Benchmark (STS-B): A collection of sentence pairs drawn from news headlines and other sources  annotated with a score from 1 to 5 denoting how similar the two sentences are in terms of semantic meaning.

7. Microsoft Research Paraphrase Corpus (MRPC): Sentence pairs automatically extracted from online news sources, with human annotations for whether the sentences in the pair are semantically equivalent.

8. Recognizing Textual Entailment (RTE): Similar to MNLI


### BERT as a word embedding method

**Umm, can we just use the output representation vectors as word embeddings?**

Similar to what we had with `ELMo`, we can also use the output vector representations as embeddings for corresponding input token, no fine-tune needed.

<img src="http://jalammar.github.io/images/bert-contexualized-embeddings.png" alt="BERT for word embedding" width="80%"/>

The question is, should we just use the last layer hidden vectors solely? (i.e. the output vectors)

Or could we make use of some of the hidden layer vectors as well?

There are many options, I mean an INFINITE number of options! Concatenation, summation, linear combination, with any choice of vectors. But the paper examines six of them for the NER task.

<img src="http://jalammar.github.io/images/bert-feature-extraction-contextualized-embeddings.png" alt="BERT for word embedding combinations" width="80%"/>


Last but not least, Google has open sourced the [BERT model on their Github page](https://github.com/google-research/bert).

#### Sample Code?

There is a great sample code on [Google Colab](https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb) by Jay Alammar that is a great start for fine tuning and getting BERT work for downstream tasks without much effort. It has great explanation as well as self-explainatory comments.